In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import sys
import os

In [2]:
from brian2 import *
from brian2tools import *

import matplotlib.pyplot as plt

import cx_rate
import trials
import plotter

from cx_spiking.constants import *

import cx_spiking.plotting
import cx_spiking.inputs
import cx_spiking.network_creation as nc

import cx_spiking.optimisation.metric as metric
import cx_spiking.optimisation.ng_optimiser as ng_optimiser

### Load route and generate spike versions of headings and optic flow

In [3]:
route_file = 'data/route.npz'
T_outbound = 1500

h, v, = cx_spiking.inputs.generate_route(T_outbound=1500, vary_speed=True, route_file=route_file, load_route=True)

cx_spiking.inputs.save_route(route_file, h, v, save_route=True)

# Convert headings
headings, digitized = cx_spiking.inputs.compute_headings(h, N=N_TL2//2, vmin=5, vmax=100)
headings = np.tile(headings, 2)

# Convert velocity into optical flow
flow = cx_spiking.inputs.compute_flow(h, v, baseline=50, vmin=0, vmax=50)

Load route from data/route.npz
data/route.npz exists - not overwriting it


### Rate based CX

In [4]:
noise = 0.1
cx = cx_rate.CXRatePontin(noise=noise)

h, v, log, cpu4_snapshot = trials.run_trial(logging=True,
                                            T_outbound=T_outbound,
                                            T_inbound=0,
                                            noise=noise,
                                            cx=cx,
                                            route=(h[:T_outbound], v[:T_outbound]))

### Poisson groups

In [5]:
start_scope()

time_step = 20 # ms

This is the spiking input

In [6]:
h_stimulus = TimedArray(headings*Hz, dt=1.*time_step*ms)
P_HEADING = PoissonGroup(N_TL2, rates='h_stimulus(t,i)')

Model neuron group that requires optimisation.

In [7]:
# Neuron group
G_TL2 = nc.generate_neuron_groups(N_TL2, eqs, threshold_eqs, reset_eqs, params, name='TL2')

# Add monitors
STM_TL2, SPM_TL2 = nc.add_monitors(G_TL2, name='TL2')

# Connect heading to TL2
S_P_HEADING_TL2 = nc.connect_synapses(P_HEADING, G_TL2, W_HEADING_TL2, model=synapses_model, on_pre=synapses_eqs_ex)

This is the **target** for the optimisation.

In [8]:
TL2_spike_rates = 90 # Hz

# Scale spike rates from rate-based CX in the right range
# transpose since log is neuron_index*time_step but we want the opposite
TL2_stimulus = TimedArray(TL2_spike_rates*log.tl2.T*Hz, dt=1.*time_step*ms)
P_TL2 = PoissonGroup(N_TL2, rates='TL2_stimulus(t,i)')
SPM_TL2_IDEAL = SpikeMonitor(P_TL2, name='TL2_target')

Store network 

In [9]:
store()

In [10]:
def run_simulation(tauE, tauI, wE, wI, G, target_spike_monitor,
                   time, dt_, delta, rate_correction): 
    restore() 
    
    # set the parameters 
    tauE = tauE * ms
    tauI = tauI * ms

    wE = wE * nS
    wI = wI * nS
    
    _, model_spike_monitor = nc.add_monitors(G)
    
    run(time)
    
    gf = metric.compute_gamma_factor(model_spike_monitor, target_spike_monitor, time, 
                                     dt_=dt_, delta=delta, rate_correction=rate_correction)

    return gf

In [11]:
# Values to optimise
bounds = [[0,5],     # tauE [ms]
          [0,5],     # tauI [ms]
          [200,1000], # wE   [nS]
          [200,1000]] # wI   [nS]


# Other fixed arguments to optimisation function
delta = 1*ms
rate_correction = True

args = [
        G_TL2,                   # neuron group to optimise
        SPM_TL2_IDEAL,           # target spike monitor
        T_outbound*time_step*ms, # simulation time
        defaultclock.dt,         # simulation time step
        delta,                   # time window for gamma factor
        rate_correction          # apply rate correction to gamma factor
       ]

In [12]:
# Set instruments
instruments = ng_optimiser.set_instrumentation(bounds, args)
optim = ng_optimiser.set_optimiser(instruments, method='DE', budget=10)

WARNING    /Users/ff/.pyenv/versions/3.8.2/envs/py_msc/lib/python3.8/site-packages/nevergrad/optimization/differentialevolution.py:216: InefficientSettingsWarning: DE algorithms are inefficient with budget < 60
  warnings.warn("DE algorithms are inefficient with budget < 60", base.InefficientSettingsWarning)
 [py.warnings]
  warnings.warn("DE algorithms are inefficient with budget < 60", base.InefficientSettingsWarning)



[Array((1,), [ArctanBound(a_max=[5], a_min=[0], name=At([0],[5]), shape=(1,))]), Array((1,), [ArctanBound(a_max=[5], a_min=[0], name=At([0],[5]), shape=(1,))]), Array((1,), [ArctanBound(a_max=[1000], a_min=[200], name=At([200],[1000]), shape=(1,))]), Array((1,), [ArctanBound(a_max=[1000], a_min=[200], name=At([200],[1000]), shape=(1,))]), NeuronGroup(clock=Clock(dt=100. * usecond, name='defaultclock'), when=start, order=0, name='TL2'), <SpikeMonitor, recording from TL2_target>, 30. * second, 100. * usecond, 1. * msecond, True]


In [13]:
optim_min, recommendation = ng_optimiser.run_optimiser(optim, run_simulation, verbosity=2)

Launching 1 jobs with new suggestions


WARNING    "EL" is an internal variable of group "TL2", but also exists in the run namespace with the value -52. * mvolt. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]


Updating fitness with value 3.0213058117583143
9 remaining budget and 0 running jobs
Current pessimistic best is: Point<x: [-0.2130243  -0.05558866 -1.20891569  0.47500615], mean: 3.0213058117583143, count: 1>
Launching 1 jobs with new suggestions
Updating fitness with value 2.1451260156064933
8 remaining budget and 0 running jobs
Current pessimistic best is: Point<x: [-0.15146499 -0.13072596 -2.89135973 -1.59275687], mean: 2.1451260156064933, count: 1>
Launching 1 jobs with new suggestions
Updating fitness with value 3.212802689516736
7 remaining budget and 0 running jobs
Current pessimistic best is: Point<x: [-0.15146499 -0.13072596 -2.89135973 -1.59275687], mean: 2.1451260156064933, count: 1>
Launching 1 jobs with new suggestions
Updating fitness with value 3.927313343490125
6 remaining budget and 0 running jobs
Current pessimistic best is: Point<x: [-0.15146499 -0.13072596 -2.89135973 -1.59275687], mean: 2.1451260156064933, count: 1>
Launching 1 jobs with new suggestions
Updating f

In [15]:
optim_min.provide_recommendation()

Candidate(args=(2.2607544874580032, 2.293116352177294, 284.7926912211878, 342.76594197909304, NeuronGroup(clock=Clock(dt=100. * usecond, name='defaultclock'), when=start, order=0, name='TL2'), <SpikeMonitor, recording from TL2_target>, 30. * second, 100. * usecond, 1. * msecond, True), kwargs={}, data=[-0.15146499 -0.13072596 -2.89135973 -1.59275687])